# 3章 関数の構文

## パターンマッチ

関数の引数に具体的な値を入れることでif/else書かなくてよくなる  
具体的な値ではなく小文字から始まる名前にすると任意の値に合致するようになる

In [2]:
sayMe :: Int -> String

sayMe 1 = "one"
sayMe 2 = "two"
sayMe 3 = "three"
sayMe x = "uncool"

sayMe 2
sayMe 18

"two"

"uncool"

関数を再起的に定義することで階乗の計算もできる

In [4]:
factorial :: Int -> Int
factorial 0 = 1
factorial n = n * factorial (n-1)

factorial 3

6

### タプルのパターンマッチ

パターンマッチを知らないハダ

In [5]:
addVectors :: (Double, Double) -> (Double, Double) -> (Double, Double)
addVectors a b = (fst a + fst b, snd a + snd b)

addVectors (3.8, 4.2) (1.2, 0.5)

(5.0,4.7)

パターンマッチを知ったハダ  
x1, x2 , y1, y2はそれぞれ任意の値

In [6]:
addVectors :: (Double, Double) -> (Double, Double) -> (Double, Double)
addVectors (x1, y1) (x2, y2) = (x1 + x2, y1 + y2)

addVectors (3.8, 4.2) (1.2, 0.5)

(5.0,4.7)

トリプルのタプルに対して要素を取り出す

In [7]:
first :: (a, b, c) -> a
first (x, _, _) = x

second :: (a, b, c) -> b
second (_, y, _) = y

third :: (a, b, c) -> c
third (_, _, z) = z

first (1,2,3)
second (1,2,3)
third (1,2,3)

1

2

3

### リストのパターンマッチ

In [10]:
let xs = [(1,3),(4,3),(2,4),(5,3),(5,6),(3,1)]
[a+b | (a, b) <- xs]

[4,7,6,8,11,4]

リスト内包表記のパターンマッチは失敗したら次の要素に進むだけ

In [9]:
let xs = [(1,3),(4,3),(2,4),(5,3),(5,6),(3,1)]
[x*100+3 | (x, 3) <- xs]

[103,403,503]

In [11]:
head' :: [a] -> a
head' [] = error "not empty list"

-- [1,2,3]は1:2:3:[]の構文糖衣
head' (x:_) = x

head' [4,5,6]
head' []

4

: 

パターンマッチでは++演算子(二つのリストをつなげる)は使えない

### asパターン

パターンマッチの対象になった元の値を参照したい時に使う  
`all@(x:_)`だとallが元の値  
`"Yota"`は`['Y','o','t','a']`と一緒で`x:_`は`[x,..]`の糖衣構文

In [16]:
firstLetter :: String -> String
firstLetter "" = "Empty String"
-- firstLetter all@(x:xs) = "The first letter of " ++ all ++ " is " ++ [x]
-- xs使わないからこっちのが良さそう
firstLetter all@(x:_) = "The first letter of " ++ all ++ " is " ++ [x]

firstLetter "Yota"

"The first letter of Yota is Y"

### ガード

ガード(`|`以下の条件式)は最低でも1つのスペースが必要

In [18]:
checkBmi :: Double -> String
checkBmi bmi
    | bmi <= 18.5 = "good"
    | bmi <= 25.0 = "so so"
    | bmi <= 30.0 = "fat"
    | otherwise = "whale"
    
checkBmi 17.5
checkBmi 27
checkBmi 34

"good"

"fat"

"whale"

複数の引数にも対応

In [22]:
checkBmi :: Double -> Double -> String
checkBmi weight height
    | weight / height ^ 2 <= 18.5 = "good"
    | weight / height ^ 2 <= 25.0 = "soso"
    | weight / height ^ 2 <= 30 = "fat"
    | otherwise = "whale"
    

checkBmi 66 1.72

"soso"

中置関数のパターン

In [21]:
myCompare :: (Ord a) => a -> a-> Ordering
a `myCompare` b
    | a == b = EQ
    | a <= b = LT
    | otherwise = GT
    
myCompare 3 3
myCompare 3 6
myCompare 3 2
    

EQ

LT

GT

### where

whereキーワードを使って計算の中間結果に名前をつける

In [54]:
checkBmi :: Double -> Double -> String
checkBmi weight height
    | bmi <= 18.5 = "good"
    | bmi <= 25.0 = "soso"
    | bmi <= 30 = "fat"
    | otherwise = "whale"
    where bmi = weight / height ^ 2
    
checkBmi 66 1.72

"soso"

In [57]:
checkBmi :: Double -> Double -> String
checkBmi weight height
    | bmi <= skinny = "good"
    | bmi <= 25.0 = "soso"
    | bmi <= 30 = "fat"
    | otherwise = "whale"
    where  bmi = weight / height ^ 2
                skinny = 18.5
    
checkBmi 66 1.72

: 